In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 38.9MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 40.0MB/s eta 0:00:01
     |████████████████████████████████| 133kB 46.7MB/s eta 0:00:01
     |████████████████████████████████| 860kB 44.1MB/s eta 0:00:01
     |████████████████████████████████| 675kB 40.0MB/s eta 0:00:01
     |████████████████████████████████| 734.6MB 49kB/s s eta 0:00:01    |██▏                             | 49.8MB 43.8MB/s eta 0:00:16
     |████████████████████████████████| 5.8MB 35.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 18.1MB/s eta 0:00:01
     |████████████████████████████████| 235kB 44.0MB/s eta 0:00:01
     |████████████████████████████████| 552kB 42.6MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=da37734307b3da1f36851e1746c2b2664a6073baf1285a5026d4438faf42d1c3
  Stored in directory: /home/jovyan/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede

In [2]:
!pip install tqdm

In [3]:
!pip install torchsnooper

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D" -O 10layers_12heads_1024len_768embd_full_corpus_16bsize.zip && rm -rf /tmp/cookies.txt

--2019-12-05 09:23:30--  https://docs.google.com/uc?export=download&confirm=KDY2&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D
Resolving docs.google.com (docs.google.com)... 172.217.31.174, 2404:6800:4004:80c::200e
Connecting to docs.google.com (docs.google.com)|172.217.31.174|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cdcf75cdl5gg57br0vess68goeonrgc4/1575532800000/08564091608614806726/*/1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D?e=download [following]
--2019-12-05 09:23:30--  https://doc-0g-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cdcf75cdl5gg57br0vess68goeonrgc4/1575532800000/08564091608614806726/*/1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D?e=download
Resolving doc-0g-38-docs.googleusercontent.com (doc-0g-38-docs.googleusercontent.com)... 172.217.25.225, 2404:6800:4004:81b::2001
Connecting to doc-0g-38-docs.googleusercontent.com (doc-0g-

In [5]:
import sys

In [6]:
GITHUB_REPO = "GPT2-Chinese"

In [7]:
!rm -rf {GITHUB_REPO}
!git clone https://github.com/Morizeyao/{GITHUB_REPO}.git {GITHUB_REPO}

Cloning into 'GPT2-Chinese'...
remote: Enumerating objects: 184, done.
remote: Total 184 (delta 0), reused 0 (delta 0), pack-reused 184
Receiving objects: 100% (184/184), 13.40 MiB | 8.28 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [8]:
if not GITHUB_REPO in sys.path:
    sys.path += [GITHUB_REPO]

In [9]:
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
    sys.path += ['bertviz_repo']
!pip install regex
!pip install transformers

     |████████████████████████████████| 368kB 34.3MB/s eta 0:00:01


In [10]:
import json
import torch
import textwrap
import torchsnooper
import pytorch_transformers
import torch.nn.functional as F
from tqdm import trange
from IPython.core.display import display, HTML

In [11]:
pretrained_model = '10layers_12heads_1024len_768embd_full_corpus_16bsize'

In [12]:
!unzip {pretrained_model}.zip

Archive:  10layers_12heads_1024len_768embd_full_corpus_16bsize.zip
  inflating: home/ec2-user/SageMaker/tmp/GPT2-Chinese/model/10layers_12heads_1024len_768embd_full_corpus_16bsize/config.json  
  inflating: home/ec2-user/SageMaker/tmp/GPT2-Chinese/model/10layers_12heads_1024len_768embd_full_corpus_16bsize/pytorch_model.bin  
  inflating: home/ec2-user/SageMaker/tmp/GPT2-Chinese/cache/vocab_small.txt  


In [13]:
sagemaker_base_path = 'home/ec2-user/SageMaker/tmp/GPT2-Chinese'
config_file = 'config.json'
model_ckpt = "pytorch_model.bin"
vocab_file = "vocab_small.txt"

In [14]:
!rm {config_file} {model_ckpt} {vocab_file}

rm: cannot remove 'pytorch_model.bin': No such file or directory


In [15]:
!mv {sagemaker_base_path}/model/{pretrained_model}/{config_file} {config_file}
!mv {sagemaker_base_path}/model/{pretrained_model}/{model_ckpt} {model_ckpt}
!mv {sagemaker_base_path}/cache/{vocab_file} {vocab_file}

In [16]:
from tokenizations import tokenization_bert

In [17]:
config = pytorch_transformers.GPT2Config.from_json_file(config_file)
config.output_attentions = True

In [18]:
model = pytorch_transformers.GPT2LMHeadModel.from_pretrained(".", config=config)
tokenizer = tokenization_bert.BertTokenizer(vocab_file=vocab_file)

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(13317, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [20]:
def is_word(word):
    for item in list(word):
        if item not in 'qwertyuiopasdfghjklzxcvbnm':
            return False
    return True

In [21]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [22]:
def fast_sample_sequence(model,context,length,temperature=1, top_k=0, top_p=0.0,device='cpu'):
    inputs = torch.LongTensor(context).view(1, -1).to(device)
    if len(context) > 1:
        _, past = model(inputs[:, :-1], None)[:2]
        prev = inputs[:, -1].view(1, -1)
    else:
        past = None
        prev = inputs
    generate = [] + context
    with torch.no_grad():
        for i in trange(length):
            output = model(prev, past=past)
            output, past = output[:2]
            output = output[-1].squeeze(0) / temperature
            filtered_logits = top_k_top_p_filtering(output, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(torch.softmax(filtered_logits, dim=-1), num_samples=1)
            
            # redraw if [UNK]
            if next_token.unsqueeze(0) != 100:
                generate.append(next_token.item())
                prev = next_token.view(1, 1)

    return generate

In [23]:
def get_html(context, generated_text, novel_name='', algorithm=''):
    if generated_text[-1] != '。':
        if generated_text[-1] == '，':
            generated_text= generated_text[:-1]
        generated_text += ' ...'
    
    html = """
    <html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
    * {
      box-sizing: border-box;
    }

    /* Create two unequal columns that floats next to each other */
    .column {
      float: left;
      padding: 10px;
    }

    .left {
      width: 20%;
    }

    .right {
      width: 80%;
    }

    /* Clear floats after the columns */
    .row:after {
      content: "";
      display: table;
      clear: both;
    }
    </style>
    </head>
    <body>

    <div style="background-color:#FFFFF0 !important">
        <div class="row">
          <div class="column left">
            <h3 style="color: #bfbaba;text-align:center">
                GPT-2看了這個標題的內容：
                novel_name
                
            </h2>
          </div>
          <div class="column right">
            <h3 style="color: #708090;line-height: 1.5">context</h3>
          </div>
        </div>

        <hr/>

        <div class="row">
          <div class="column left">
            <h3 style="color: #00477D;text-align:center">
                algorithm
                GPT2說...
            </h2>
          </div>
          <div class="column right">
            <h3 style="color: #006374;line-height: 1.5">generated_text</h3>

          </div>
        </div>
    </div>

    </body>
    </html>


    """.replace('context', context).replace('generated_text', generated_text).replace("novel_name", f'《{novel_name}》<br/>')
    
    if not algorithm:
        html = html.replace("algorithm", "")
    else:
        html = html.replace("algorithm", f'{algorithm}<br/>')
    
    
    return html

In [24]:
def generate(context, topk, topp, temperature, device, line_len=40, novel_name=''):
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(context))
        
    # auto-regressive
    out = fast_sample_sequence(
        model=model, length=length,
        context=context_tokens,
        temperature=temperature, top_k=topk, top_p=topp, device=device
    )

    # rendering
    tokens = tokenizer.convert_ids_to_tokens(out)

    for i, item in enumerate(tokens):
        if item == '[MASK]':
            tokens[i] = ''
        if item == '[CLS]' or item == '[SEP]':
            tokens[i] = '\n'
    
    generated_text = ''.join(tokens).strip().replace(context, '')
    html = get_html(context, generated_text, novel_name=novel_name)
    
    return html, generated_text

In [25]:
#@ title 生成參數、Decoding 策略設定（生成溫度等）
nsamples = 1
batch_size = 1
length = model.config.n_ctx // 2

topk = 30
topp = 0
temperature = 1

In [26]:
# 改變此變數來選擇不同的前文脈絡。當然你也可以自行填加
sample_idx = 1

In [27]:
# 飛雪連天射白鹿，笑書神俠倚碧鴛。
sampled_contexts =[
    ('寶可夢 劍／盾', '任天堂年度大作《寶可夢 劍／盾》正式發表，上市前就備受訓練家們期待，根據寶可夢公司最新公告，首週全球銷售突破  600 萬，打破 Switch 新紀錄，然而玩家社群們卻是負評如潮。'),
    ('「Cybertrunk」將於《電馭叛客 2077》登場?', '行事風格向來不按牌理出牌的特斯拉汽車執行長伊隆馬斯克 (Elon Musk) 在上週的發表會上發表了一台造型相當前衛的全新車種－「The Cybertruck」。這台充滿Cyberpunk風格的全新電動車標榜著環保的特性，以及防彈的強化玻璃，即使在發表會上出了點小包，一砸就出現裂痕。然而，即使無法購買這台真車的玩家們，也許有機會在《電馭叛客 2077》中駕駛這台獨特的汽車，奔馳於遊戲中的「夜城」(Night City) 街道。'),
]

In [28]:
sample = sampled_contexts[sample_idx]
novel_name, context = sample

In [29]:
html, generated_text = generate(context, topk, topp, temperature, device, novel_name=novel_name)
html

100%|██████████| 512/512 [00:16<00:00, 30.75it/s]


'\n    <html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <style>\n    * {\n      box-sizing: border-box;\n    }\n\n    /* Create two unequal columns that floats next to each other */\n    .column {\n      float: left;\n      padding: 10px;\n    }\n\n    .left {\n      width: 20%;\n    }\n\n    .right {\n      width: 80%;\n    }\n\n    /* Clear floats after the columns */\n    .row:after {\n      content: "";\n      display: table;\n      clear: both;\n    }\n    </style>\n    </head>\n    <body>\n\n    <div style="background-color:#FFFFF0 !important">\n        <div class="row">\n          <div class="column left">\n            <h3 style="color: #bfbaba;text-align:center">\n                GPT-2看了這個標題的內容：\n                《「Cybertrunk」將於《電馭叛客 2077》登場?》<br/>\n                \n            </h2>\n          </div>\n          <div class="column right">\n            <h3 style="color: #708090;line-height: 1.5">行事風格向來不按牌理出牌的特斯拉汽車執行長伊隆馬斯克 (Elon Mu

In [30]:
generated_text

'行事風格向來不按牌理出牌的特斯拉汽車執行長伊隆馬斯克(el##onmu##sk)在上週的發表會上發表了一台造型相當前衛的全新車種－「thec##y##bert##ru##ck」。這台充滿c##y##ber##pu##nk風格的全新電動車標榜著環保的特性，以及防彈的強化玻璃，即使在發表會上出了點小包，一砸就出現裂痕。然而，即使無法購買這台真車的玩家們，也許有機會在《電馭叛客2077》中駕駛這台獨特的汽車，奔馳於遊戲中的「夜城」(nightcity)街道。\n精忠之故鰲拜專踢了一團亮，如獲滅跡，當然有人前來投靠，勢道極緩慢，守到天明時，鄰近府的門戶溝稍停，有人推門推門進去。他見進來，當下滿是鹽擔，當下滿滿滿了一副弓箭的大小門坐在凳上，每人手上都是銬，一面團黑漆，穿了一條小門，就是一副弓彈打造得精鋼熟的模樣。\n廳上眾鹽梟見了這等大聲勢，紛紛鼓掌。安提督府中老兄贊道：「韋香主，你天明，果然有刺客。」余人一聽，都是又驚又喜，又是慚愧。老三道：「這批匪徒大會聚會，你天明日到老來，是不是來了嗎？」韋小寶笑道：「倒不是有點門戶，比之我做戲，這個夢想做戲的會唱戲，太也老是纏夾不牢，沒想要犯罪。這叫做老成，是擺明瞭料得很。」\n忽聽得廳外有人壓低聲說道：「進來。」諸鹽梟響聲中，一齊擁進，大家跳上桌來，瞧著「賊廝鳥」的模樣，有些手腳都拿著鬼頭刀。大家眼見那些鹽商人平日之事不知聞。\n韋小寶震於鹽稅甚多，討了些錢鹽稅，弄得本錢生意，十幾只鹽稅一濫，沒多幾便加一錢稅，何必多結幾條人命？突然見有聲音一變，不由得臉上變色，但大吃一驚，這一驚非同小可。韋小寶急忙跳起身來，噗的一聲，坐入房中，從掌櫃台踢了一腳，大叫：「拿酒來，拿酒來！」\n他一鑽入房中，立即縮頭不見有人，立即縮頭走出，只聽那'

In [31]:
display(HTML(html))

## 視覺化自注意力機制

In [32]:
#@title BertViz wrapper
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
      '''))

In [33]:
from bertviz import head_view as show_head
from bertviz import model_view as show_model
from bertviz import neuron_view as show_neuron
from transformers import BertTokenizer, BertModel
from transformers import GPT2Tokenizer, GPT2Model

In [34]:
#from bertviz.pytorch_transformers_attn import BertModel, BertTokenizer
#from bertviz.head_view_bert import show

In [35]:
#from bertviz.pytorch_transformers_attn import GPT2Model
#from bertviz.head_view import show as show_head
#from bertviz.model_view import show as show_model
#from bertviz.neuron_view import show as show_neuron

In [36]:
def call_html(view):
    import IPython
    if view in ['model', 'neuron']:
        display(IPython.core.display.HTML('''
             <script src="/static/components/requirejs/require.js"></script>
             <script>
               requirejs.config({
                 paths: {
                   base: '/static/base',
                   "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
                   jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
                 },
               });
             </script>
        '''))
    else:
        display(IPython.core.display.HTML('''
             <script src="/static/components/requirejs/require.js"></script>
             <script>
               requirejs.config({
                 paths: {
                   base: '/static/base',
                   "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
                   jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
                 },
               });
             </script>
       '''))

In [37]:
'''不用了'''
'''
def show(model, tokenizer, text, view, model_type='gpt2'):
    call_html(view)
    show_func = {'head': show_head, 'model': show_model, 'neuron': show_neuron}
    show_func[view](model, model_type, tokenizer, text)
    '''

"\ndef show(model, tokenizer, text, view, model_type='gpt2'):\n    call_html(view)\n    show_func = {'head': show_head, 'model': show_model, 'neuron': show_neuron}\n    show_func[view](model, model_type, tokenizer, text)\n    "

In [38]:
'''不用了'''
'''
def show(model, tokenizer, text, view, model_type='gpt2'):
    call_html(view)
    model_version = 'gpt2'
    model = GPT2Model.from_pretrained(model_version, output_attentions=True)
    tokenizer = GPT2Tokenizer.from_pretrained(model_version)
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    show_func = {'head': show_head, 'model': show_model, 'neuron': show_neuron}
    show_func[view](attention, tokens)
    '''

"\ndef show(model, tokenizer, text, view, model_type='gpt2'):\n    call_html(view)\n    model_version = 'gpt2'\n    model = GPT2Model.from_pretrained(model_version, output_attentions=True)\n    tokenizer = GPT2Tokenizer.from_pretrained(model_version)\n    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)\n    input_ids = inputs['input_ids']\n    token_type_ids = inputs['token_type_ids']\n    attention = model(input_ids, token_type_ids=token_type_ids)[-1]\n    input_id_list = input_ids[0].tolist() # Batch index 0\n    tokens = tokenizer.convert_ids_to_tokens(input_id_list)\n    show_func = {'head': show_head, 'model': show_model, 'neuron': show_neuron}\n    show_func[view](attention, tokens)\n    "

In [39]:
def show(text, view, model_type='gpt2'):
    call_html(view)
    model_version = 'gpt2'
    model = GPT2Model.from_pretrained(model_version, output_attentions=True)
    model.to('cpu')
    tokenizer = GPT2Tokenizer.from_pretrained(model_version)
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    show_func = {'head': show_head, 'model': show_model, 'neuron': show_neuron}
    show_func[view](attention, tokens)

In [40]:
def show_head_view(text, model_type='gpt2'):
    call_html(view)
    model_version = 'gpt2'
    model = GPT2Model.from_pretrained(model_version, output_attentions=True)
    model.to('cpu')
    tokenizer = GPT2Tokenizer.from_pretrained(model_version)
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    show_head(attention, tokens)

In [41]:
def show_model_view(text, model_type='gpt2'):
    call_html(view)
    model_version = 'gpt2'
    model = GPT2Model.from_pretrained(model_version, output_attentions=True)
    model.to('cpu')
    tokenizer = GPT2Tokenizer.from_pretrained(model_version)
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    show_model(attention, tokens)

In [42]:
#gpt2_model = GPT2Model.from_pretrained('.')
#gpt2_model.to('cpu')
#gpt2_model.eval()

## 選擇視覺化方式

In [43]:
#@title head 模式
#text = u'小明今天沒來，聽說他生病了'
text = 'Mary at the store, she bought apples, oranges, bananas,'

In [44]:
print(text)

Mary at the store, she bought apples, oranges, bananas,


In [45]:
#how(gpt2_model, tokenizer, text, view)

In [46]:
show_head_view(text)

NameError: name 'view' is not defined

In [ ]:
#@title model 模式
show_model_view(text)

In [ ]:
#@title neuron 模式

#view = 'neuron'
#show(text, view)

In [ ]:
from bertviz.transformers_neuron_view import GPT2Model, GPT2Tokenizer
from bertviz.neuron_view import show as show_neuron

In [ ]:
model_type = 'gpt2'
model_version = 'gpt2'
model = GPT2Model.from_pretrained(model_version)
tokenizer = GPT2Tokenizer.from_pretrained(model_version)

In [ ]:
show_neuron(model, model_type, tokenizer, text)